In [1]:
import os
import re
import logging
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from keras import initializers, regularizers, constraints
from keras.preprocessing import sequence
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Sequential, load_model, Model
from keras.layers import (
    Dense,
    Dropout,
    Activation,
    MaxPooling1D,
    Flatten,
    Embedding,
    Conv1D,
    GlobalMaxPooling1D,
    BatchNormalization,
    CuDNNGRU,
    LSTM,
    Bidirectional,
    Input,
    concatenate,
)
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Nadam
from keras.utils.vis_utils import plot_model
from keras import backend as K
from numpy import random
import gensim
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations


In [2]:
df = pd.read_csv('olddata/reviewContent+metadata.csv', encoding = "ISO-8859-1")

c:\Users\prems\anaconda3\envs\swm\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=df.dropna() 

In [4]:
df

,user_id,prod_id,rating,label,date,text_
0,5044.0,0.0,1.0,-1.0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045.0,0.0,1.0,-1.0,9/8/2014,This was the worst experience I've ever had a ...
2,5046.0,0.0,3.0,-1.0,10/6/2013,This is located on the site of the old Spruce ...
3,5047.0,0.0,5.0,-1.0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048.0,0.0,5.0,-1.0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
608593,119664.0,5039.0,4.0,1.0,1/20/2013,When I first moved to the area I must say I wa...
608594,56277.0,5039.0,2.0,1.0,11/12/2012,Kind of pricey. I guess I expected a ridiculou...
608595,265320.0,5039.0,1.0,1.0,8/22/2012,"Stopped by this restaurant yesterday, we just ..."
608596,161722.0,5039.0,4.0,1.0,5/11/2011,Finally checked out The Best Subs in Claremont...


In [5]:
df.describe()


,user_id,prod_id,rating,label
count,608598.000000,608598.000000,608598.000000,608598.000000
mean,89642.634603,2564.991760,3.923595,0.735569
std,73873.157594,1447.929346,1.147472,0.677450
min,5044.000000,0.000000,1.000000,-1.000000
25%,25495.250000,1345.000000,3.000000,1.000000
50%,66770.000000,2616.000000,4.000000,1.000000
75%,142269.000000,3768.000000,5.000000,1.000000
max,265320.000000,5043.000000,5.000000,1.000000


In [6]:
# Assuming 'text_' column contains the reviews
max_review_length = df['text_'].apply(len).max()
min_review_length = df['text_'].apply(len).min()

print(f"Maximum Review Length: {max_review_length}")
print(f"Minimum Review Length: {min_review_length}")

Maximum Review Length: 5000
Minimum Review Length: 1


In [15]:
# Assuming 'text_' column contains the reviews
reviews_length_one = df[df['text_'].apply(len) == 1]['text_']

print("Reviews with Length 1:")
print(reviews_length_one)

Reviews with Length 1:
526       -
22764     a
25053     I
27779     .
29073     !
50634     .
52283     X
95367     .
107159    .
121220    J
123032    .
127073    -
147737    h
151219    .
155171    A
159153    .
163349    .
164741    .
185989    .
191799    N
194878    !
203617    f
215340    h
240496    a
241129    N
243414    .
269098    .
284220    .
295912    .
310130    .
318126    .
331864    b
374621    5
388302    .
394531    -
410264    .
411009    .
432470    .
436863    .
450002    .
473665    .
496995    A
498465    .
500825    .
506167    .
519394    .
522682    I
543742    .
552760    a
559335    ?
578877    .
579763    .
591571    .
593161    .
Name: text_, dtype: object


In [8]:
len(df['prod_id'].unique())

5044

In [16]:

# Function to calculate Maximum Number of Reviews (MNR)
def calculate_mnr(df1):
    df1['date'] = pd.to_datetime(df1['date'])
    df1['reviews_per_day'] = df1.groupby(['user_id', df1['date'].dt.date])['user_id'].transform('count')
    max_reviews_per_day = df1.groupby('user_id')['reviews_per_day'].max()
    return max_reviews_per_day

# Function to calculate Percentage of Positive Reviews (PR)
def calculate_pr(df1):
    # Group reviews by user_id and create a list of reviews for each user
    user_reviews = df1.groupby('user_id')['rating'].apply(list).reset_index(name='reviews')

    # Calculate the percentage of positive reviews for each user
    user_reviews['pr'] = user_reviews['reviews'].apply(
        lambda reviews: (
            sum(rating in [4, 5] for rating in reviews) / len(reviews)
        ) if len(reviews) > 0 else 0
    )

    # Merge the result back to the original dataframe
    pr_df = user_reviews[['user_id', 'pr']]
    return pr_df

# Function to calculate Review Length (RL)
def calculate_rl(df1):
    df1['review_length'] = df1['text_'].apply(lambda x: len(x.split()))
    avg_review_length = df1.groupby('user_id')['review_length'].mean()
    return avg_review_length

# Function to calculate Reviewer Deviation (RD)
def calculate_rd(df1):
    average_ratings_per_product = df1.groupby('prod_id')['rating'].mean()
    tempdf = pd.merge(df1, average_ratings_per_product, on='prod_id', how='left')
    tempdf['rating_deviation'] = np.abs(tempdf['rating_x'] - tempdf['rating_y'])
    average_deviation_per_user = tempdf.groupby('user_id')['rating_deviation'].mean()
    return average_deviation_per_user

# Function to calculate Maximum Content Similarity (MCS)
def calculate_mcs(df1):
    # Group by user_id and collect all reviews
    user_reviews = df1.groupby('user_id')['text_'].agg(lambda x: list(map(str, x))).reset_index()

    # Initialize lists to store user_ids and max similarities
    user_ids = []
    max_similarities = []
    user_reviews_dict = dict(zip(user_reviews['user_id'], user_reviews['text_']))



    # Create a TF-IDF vectorizer outside the loop
    vectorizer = TfidfVectorizer(stop_words=None)

    for user_id, reviews in user_reviews_dict.items():
        # If there is only one review, similarity is 0
        if len(reviews) <= 1:
            user_ids.append(user_id)
            max_similarities.append(0)
        else:
            reviews = list(map(str, reviews))
            try:
                # Calculate TF-IDF vectors and cosine similarity between all pairs of reviews
                tfidf_matrix = vectorizer.fit_transform(reviews)
                similarity_matrix = cosine_similarity(tfidf_matrix)

                # Set the diagonal elements to 0 to avoid self-comparison
                np.fill_diagonal(similarity_matrix, 0)

                # Find the maximum similarity value
                max_similarity = np.max(similarity_matrix)
                
                # Append user_id and corresponding maximum similarity
                user_ids.append(user_id)
                max_similarities.append(max_similarity)
            except Exception as e:
                print("these reviews failed : ", user_id, reviews)
                user_ids.append(user_id)
                max_similarities.append(0)
    return pd.DataFrame({'user_id': user_ids, 'mcs': max_similarities})

In [17]:
df['text_'].dropna(inplace=True)

In [18]:
copy_df=df.copy()

In [19]:
df

,user_id,prod_id,rating,label,date,text_
0,5044.0,0.0,1.0,-1.0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045.0,0.0,1.0,-1.0,9/8/2014,This was the worst experience I've ever had a ...
2,5046.0,0.0,3.0,-1.0,10/6/2013,This is located on the site of the old Spruce ...
3,5047.0,0.0,5.0,-1.0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048.0,0.0,5.0,-1.0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
608593,119664.0,5039.0,4.0,1.0,1/20/2013,When I first moved to the area I must say I wa...
608594,56277.0,5039.0,2.0,1.0,11/12/2012,Kind of pricey. I guess I expected a ridiculou...
608595,265320.0,5039.0,1.0,1.0,8/22/2012,"Stopped by this restaurant yesterday, we just ..."
608596,161722.0,5039.0,4.0,1.0,5/11/2011,Finally checked out The Best Subs in Claremont...


In [20]:

print("Calculating MNR...")
mnr_df = calculate_mnr(copy_df)
print("MNR calculation completed.")

print("Calculating PR...")
pr_df = calculate_pr(copy_df)
print("PR calculation completed.")

print("Calculating RL...")
rl_df = calculate_rl(copy_df)
print("RL calculation completed.")

print("Calculating RD...")
rd_df = calculate_rd(copy_df)
print("RD calculation completed.")

print("Calculating MCS...")
mcs_df = calculate_mcs(copy_df)
print("MCS calculation completed.")



Calculating MNR...
MNR calculation completed.
Calculating PR...
PR calculation completed.
Calculating RL...
RL calculation completed.
Calculating RD...
RD calculation completed.
Calculating MCS...
these reviews failed :  86744.0 ['.', '.', '.']
these reviews failed :  88926.0 ['-', '-']
MCS calculation completed.


In [21]:
df

,user_id,prod_id,rating,label,date,text_
0,5044.0,0.0,1.0,-1.0,11/16/2014,"Drinks were bad, the hot chocolate was watered..."
1,5045.0,0.0,1.0,-1.0,9/8/2014,This was the worst experience I've ever had a ...
2,5046.0,0.0,3.0,-1.0,10/6/2013,This is located on the site of the old Spruce ...
3,5047.0,0.0,5.0,-1.0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...
4,5048.0,0.0,5.0,-1.0,8/28/2014,I love Toast! The food choices are fantastic -...
...,...,...,...,...,...,...
608593,119664.0,5039.0,4.0,1.0,1/20/2013,When I first moved to the area I must say I wa...
608594,56277.0,5039.0,2.0,1.0,11/12/2012,Kind of pricey. I guess I expected a ridiculou...
608595,265320.0,5039.0,1.0,1.0,8/22/2012,"Stopped by this restaurant yesterday, we just ..."
608596,161722.0,5039.0,4.0,1.0,5/11/2011,Finally checked out The Best Subs in Claremont...


In [22]:
balanced_df = pd.read_csv('reviewContent+metadataBalanced.csv')

In [23]:

# Merge the calculated features with the original dataset on 'user_id'
balanced_df = balanced_df.merge(mnr_df, on='user_id', how='left')
balanced_df = balanced_df.merge(pr_df, on='user_id', how='left')
balanced_df = balanced_df.merge(rl_df, on='user_id', how='left')
balanced_df = balanced_df.merge(rd_df, on='user_id', how='left')
balanced_df = balanced_df.merge(mcs_df, on='user_id', how='left')

In [25]:
balanced_df = balanced_df.rename(columns={'reviews_per_day': 'maximum_review_per_day_for_user', 'pr': 'percent_postive_reviews_by_user','review_length': 'max_user_review_length','rating_deviation': 'avg_user_rating_deviation_per_product','mcs': 'max_cosine_similarity_to_user_reviews'})

In [26]:
balanced_df

,user_id,prod_id,rating,label,date,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
0,5044,0,1,0,11/16/2014,"Drinks were bad, the hot chocolate was watered...",1,0.000000,36.000000,2.613636,0.000000
1,5045,0,1,0,9/8/2014,This was the worst experience I've ever had a ...,1,0.000000,248.000000,2.613636,0.000000
2,5046,0,3,0,10/6/2013,This is located on the site of the old Spruce ...,1,0.250000,45.250000,0.588522,0.322332
3,5047,0,5,0,11/30/2014,I enjoyed coffee and breakfast twice at Toast ...,1,1.000000,233.000000,1.386364,0.000000
4,5048,0,5,0,8/28/2014,I love Toast! The food choices are fantastic -...,1,1.000000,152.000000,1.386364,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
160927,265253,5041,5,0,11/15/2011,Great local lounge spot! After work and regula...,1,1.000000,28.000000,0.828125,0.000000
160928,265316,5042,5,0,12/24/2014,I have been to this place for the third time i...,1,1.000000,110.000000,0.000000,0.000000
160929,62901,5043,1,0,8/4/2011,Horrible service. I saw a customer accidentall...,11,0.916667,25.416667,0.822530,0.204057
160930,201246,5043,2,0,4/14/2013,Its your standard late-night drunk filled coll...,1,0.000000,69.000000,0.675000,0.000000


In [27]:
df_no_meta = balanced_df.drop(["user_id", "prod_id","date"], axis=1)


In [28]:
df_no_meta1 =df_no_meta.sample(frac=1,random_state=1)

In [29]:
df_no_meta1.head()

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,Best Italian shop in the area. I am a NYC tran...,2,0.500000,37.000000,1.636018,0.156496
103402,4,0,Have been going here for years. Food is always...,1,1.000000,35.000000,0.188327,0.000000
160285,1,0,I was expecting more from this place. I only e...,1,0.000000,66.000000,2.355705,0.000000
99317,4,0,I got take out food here recently and was plea...,1,1.000000,59.000000,0.153061,0.000000
132540,3,1,I've been going here since high school and to ...,5,0.619048,134.380952,0.832195,0.396967


In [30]:

label_encoder = preprocessing.LabelEncoder() 

df_no_meta1['label']= label_encoder.fit_transform(df_no_meta1['label']) 

df_no_meta1['label'].unique() 

array([0, 1], dtype=int64)

In [31]:
df_no_meta1

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,Best Italian shop in the area. I am a NYC tran...,2,0.500000,37.000000,1.636018,0.156496
103402,4,0,Have been going here for years. Food is always...,1,1.000000,35.000000,0.188327,0.000000
160285,1,0,I was expecting more from this place. I only e...,1,0.000000,66.000000,2.355705,0.000000
99317,4,0,I got take out food here recently and was plea...,1,1.000000,59.000000,0.153061,0.000000
132540,3,1,I've been going here since high school and to ...,5,0.619048,134.380952,0.832195,0.396967
...,...,...,...,...,...,...,...,...
73349,5,1,The food is so fresh! Love it!,1,1.000000,7.000000,0.924862,0.000000
109259,4,0,More than a sports bar. Good food and service.,3,0.750000,15.500000,0.547648,0.178596
50057,4,1,One of those restaurants where you truly feel ...,3,0.636364,68.090909,0.729668,0.241765
5192,4,1,This place is cute! I went here long time ago....,2,0.666667,75.333333,0.828831,0.383445


In [32]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    x = str(x)
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x

In [33]:
def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]{4}', '####', x)
        x = re.sub('[0-9]{3}', '###', x)
        x = re.sub('[0-9]{2}', '##', x)
    return x

In [34]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_data(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    return text
    
df_no_meta1['text_'] = df_no_meta1['text_'].apply(clean_data)

c:\Users\prems\anaconda3\envs\swm\lib\site-packages\ipykernel_launcher.py:11: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  # This is added back by InteractiveShellApp.init_path()


In [35]:
df_no_meta1

,rating,label,text_,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews
96371,5,0,best italian shop in the area i am a nyc trans...,2,0.500000,37.000000,1.636018,0.156496
103402,4,0,have been going here for years food is always ...,1,1.000000,35.000000,0.188327,0.000000
160285,1,0,i was expecting more from this place i only ex...,1,0.000000,66.000000,2.355705,0.000000
99317,4,0,i got take out food here recently and was plea...,1,1.000000,59.000000,0.153061,0.000000
132540,3,1,i have been going here since high school and t...,5,0.619048,134.380952,0.832195,0.396967
...,...,...,...,...,...,...,...,...
73349,5,1,the food is so fresh ! love it !,1,1.000000,7.000000,0.924862,0.000000
109259,4,0,more than a sports bar good food and service,3,0.750000,15.500000,0.547648,0.178596
50057,4,1,one of those restaurants where you truly feel ...,3,0.636364,68.090909,0.729668,0.241765
5192,4,1,this place is cute ! i went here long time ago...,2,0.666667,75.333333,0.828831,0.383445


In [36]:
# X = X.map(lambda a: clean_data(a))


In [37]:
X_text = df_no_meta1.text_
X_numeric = df_no_meta1[['maximum_review_per_day_for_user','percent_postive_reviews_by_user','max_user_review_length','avg_user_rating_deviation_per_product','max_cosine_similarity_to_user_reviews','rating']]
y = df_no_meta1.label

In [38]:
x_train_text, x_test_text, x_train_numeric, x_test_numeric, y_train, y_test = train_test_split(
    X_text, X_numeric, y, stratify=y, test_size=0.2, random_state=1
)

In [39]:
tokenizer = Tokenizer(num_words=None,lower=True,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',split=' ',char_level=False)

In [40]:
tokenizer.fit_on_texts(X_text)


In [41]:
x_train_text = tokenizer.texts_to_sequences(x_train_text)

In [42]:
x_test_text = tokenizer.texts_to_sequences(x_test_text)

In [43]:
word_index = tokenizer.word_index

In [44]:
X_text = tokenizer.texts_to_sequences(X_text)

In [45]:
vocab_size = len(word_index)
print('Vocab size: {}'.format(vocab_size))
longest = max(len(seq) for seq in X_text)
print("Longest comment size: {}".format(longest))
average = np.mean([len(seq) for seq in X_text])
print("Average comment size: {}".format(average))
stdev = np.std([len(seq) for seq in X_text])
print("Stdev of comment size: {}".format(stdev))
max_len = int(average + stdev * 3)
print('Max comment size: {}'.format(max_len))

Vocab size: 87775
Longest comment size: 995
Average comment size: 105.61196033107151
Stdev of comment size: 102.15050913314423
Max comment size: 412


In [46]:
processed_post_x_train = pad_sequences(x_train_text, maxlen=max_len, padding='post', truncating='post')
processed_post_x_test = pad_sequences(x_test_text, maxlen=max_len, padding='post', truncating='post')

In [47]:
processed_x_train = pad_sequences(x_train_text, maxlen=max_len)
processed_x_test = pad_sequences(x_test_text, maxlen=max_len)

In [48]:
processed_pre_x_train = pad_sequences(x_train_text, maxlen=max_len)
processed_pre_x_test = pad_sequences(x_test_text, maxlen=max_len)

In [49]:
print('x_train shape:', processed_x_train.shape)
print('x_test shape:', processed_x_test.shape)


x_train shape: (128745, 412)
x_test shape: (32187, 412)


In [50]:
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [51]:
embedding_dim = 100
k = 0
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        k += 1
        embedding_matrix[i] = embedding_vector

In [52]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [55]:
def get_model():
    review_input = Input(shape=(max_len,), dtype='int32')
    review_input_post = Input(shape=(max_len,), dtype='int32')
    mnr_input = Input(shape=(1,), dtype='float32')  # Add MNR as a numeric input
    pr_input = Input(shape=(1,), dtype='float32')   # Add PR as a numeric input
    rl_input = Input(shape=(1,), dtype='float32')   # Add RL as a numeric input
    rd_input = Input(shape=(1,), dtype='float32')   # Add RD as a numeric input
    mcs_input = Input(shape=(1,), dtype='float32')  # Add MCS as a numeric input
    rating = Input(shape=(1,), dtype='float32')  # Add MCS as a numeric input

    x1 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input)
    x1 = Bidirectional(LSTM(60, return_sequences=True))(x1)
    x1 = Dropout(0.3)(x1)
    x1 = Attention(max_len)(x1)

    x2 = Embedding(vocab_size + 1, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(review_input_post)
    x2 = Bidirectional(LSTM(60, return_sequences=True))(x2)
    x2 = Dropout(0.3)(x2)
    x2 = Attention(max_len)(x2)

    x = concatenate([x1, x2, mnr_input, pr_input, rl_input, rd_input, mcs_input,rating])
    x = Dense(50, activation='relu')(x)
    x= Dropout(0.2)(x)
    x = BatchNormalization()(x)
    preds = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[review_input, review_input_post, mnr_input, pr_input, rl_input, rd_input, mcs_input,rating], outputs=preds)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [56]:
K._get_available_gpus()

['/device:GPU:0']

In [57]:
model = get_model()


In [58]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 412)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 412)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 412, 100)     8777600     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 412, 100)     8777600     input_2[0][0]                    
______________________________________________________________________________________________

In [59]:
from keras.callbacks import ModelCheckpoint


In [60]:
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [61]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=10)

In [62]:
x_train_numeric


,maximum_review_per_day_for_user,percent_postive_reviews_by_user,max_user_review_length,avg_user_rating_deviation_per_product,max_cosine_similarity_to_user_reviews,rating
130489,1,1.00000,42.50000,0.152120,0.214467,4
128304,1,0.00000,9.00000,1.837278,0.000000,2
130650,1,1.00000,43.50000,0.160822,0.117143,4
150573,1,1.00000,25.00000,0.869048,0.000000,5
78317,2,1.00000,38.50000,0.760401,0.100903,4
...,...,...,...,...,...,...
123085,1,1.00000,58.00000,1.120000,0.000000,5
45654,4,0.71875,110.71875,0.595595,0.375990,3
51873,5,0.80000,119.60000,0.544865,0.420381,4
151097,1,0.00000,34.00000,1.384615,0.000000,1


In [63]:
history = model.fit([processed_x_train,processed_pre_x_train,x_train_numeric['maximum_review_per_day_for_user'],x_train_numeric['percent_postive_reviews_by_user'],x_train_numeric['max_user_review_length'],x_train_numeric['avg_user_rating_deviation_per_product'],x_train_numeric['max_cosine_similarity_to_user_reviews'],x_train_numeric['rating']],y_train, validation_data=([processed_x_test,processed_pre_x_test,x_test_numeric['maximum_review_per_day_for_user'],x_test_numeric['percent_postive_reviews_by_user'],x_test_numeric['max_user_review_length'],x_test_numeric['avg_user_rating_deviation_per_product'],x_test_numeric['max_cosine_similarity_to_user_reviews'],x_test_numeric['rating']],y_test), epochs=10,batch_size=32,callbacks=[early_stopping_monitor],verbose=1)


Epoch 1/10
4024/4024 [==============================] - 463s 112ms/step - loss: 0.5451 - accuracy: 0.7293 - val_loss: 0.4809 - val_accuracy: 0.7732
Epoch 2/10
4024/4024 [==============================] - 436s 108ms/step - loss: 0.4452 - accuracy: 0.7961 - val_loss: 0.4800 - val_accuracy: 0.7630
Epoch 3/10
4024/4024 [==============================] - 433s 108ms/step - loss: 0.3900 - accuracy: 0.8264 - val_loss: 0.5352 - val_accuracy: 0.7530
Epoch 4/10
4024/4024 [==============================] - ETA: 0s - loss: 0.3473 - accuracy: 0.8476